Fill in any place that says `# YOUR CODE HERE` or YOUR ANSWER HERE, as well as your name and collaborators below.
Grading for pre-lecture assignments is all or nothing. Partial credit is available for in-class assignments and checkpoints, but **only when code is commented**.

In [ ]:
NAME = ""
COLLABORATORS = ""

---

# Learning Objectives

This lecture will show you how to:
1. Solve ODEs using an adaptive step size
2. Use `scipy.inegrate.solve_ivp`
3. Identify and solve stiff equations, and ensure that your solution conserves energy

In [ ]:
# imports
import numpy as np
import matplotlib.pyplot as plt

# from last lecture
def RK4(f, x, t, h):
    """
    Given a function f(x,t) and scalars x, t, and h, return x(t+h)
    """
    k1 = h*f(x, t)
    k2 = h*f(x + k1/2, t + h/2)
    k3 = h*f(x + k2/2, t + h/2)
    k4 = h*f(x + k3, t + h)
    return x + (k1 + 2*k2 + 2*k3 + k4)/6

from scipy import integrate # for solve_ivp

import grading_helper as _test

# Adaptive Step Size

In [ ]:
%video 7eUyxD9f3dE

Summary:

Some problems are best approached with a variable step size $h$. Set an acceptable **target accuracy** for $x$, denoted $\delta$ (measured in units of $x/t$).

**Recipe**
1. Choose a very small starting $h$ (you want this first step to be accurate).
2. Take two steps using $h$, and one step using $2h$. These let you calculate an error

 $$\epsilon=\tfrac{1}{30}\big|x_{h+h}-x_{2h}\big|\,.$$
 
3. Evaluate

 $$\rho \equiv \frac{h\delta}{\epsilon}\,.$$

    - If $\rho \ge 1$, keep $x=x_{h+h}$ from the pair of steps using $h$ (which is always better than the single step of $2h$). Calculate the next (larger) value of $h$ using
    
    $$h^\prime=h\rho^{1/4}\,.$$
    
     **Exception:** Don't let $h^\prime$ exceed $2h$. Letting $h\to\infty$ is bad.
     
   - If $\rho < 1$, throw away this step and try again with a smaller value of $h$:
    
     $$h^\prime=h\rho^{1/4}\,.$$
     
    
4. Repeat from step 2 until done. Each step, the time $t$ grows by $2h$. You don't know which values of time will be used until you calculate them.


**Extension to Simultaneous Equations:** With multiple dependent variables, you need to be more careful with the error estimate $\epsilon$. Sometimes you don't care about the accuracy of some variables, so you can leave them out. If you do care, you should calculate the error of each and add them in quadrature.

## Your Turn


Apply the adaptive step size method to the equation

$$\frac{dx}{dt} = x\cos t\,,$$

with $x(0) = 1$, and a target accuracy $\delta = 1\times10^{-5}$. Plot $x(t)$ for $0 \le t < 2\pi$. Store the $x$ values in an array named `x`, and the $t$ values in an array named `t`.

In [ ]:
%%graded # 2 points

# for comparison
t_exact = np.linspace(0, 2*np.pi, 100)
x_exact = np.exp(np.sin(t_exact))
plt.plot(t_exact, x_exact)

# YOUR CODE HERE

In [ ]:
%%tests

_test.similar(x, np.exp(np.sin(t)))
_test.plot_shown()

# Using `scipy.integrate.solve_ivp`

In [ ]:
%video INBu1Pyj0Is

Summary:

- The function `scipy.integrate.solve_ivp(f, limits, initial_values)` is a wrapper around five different ODE solvers. The default method uses the 4th order Runge-Kutta (with some extra extrapolation that effectively makes it 5th order).
- By default, the ODE is evaluated at as few points as possible. This choice is fast, but makes for lousy graphs. Instead, you will want to set `t_eval` to a sequence of points you want solutions for.

## Your Turn

The [Belousov-Zhabotinsky reaction](https://www.youtube.com/watch?v=uWh8reiXq58) is a chemical oscillator, described by the theoretical model called the "Brusselator." The equations describing the Brusselator are

$$\frac{dx}{dt} = 1 - (b+1)x + ax^2y, \qquad \frac{dy}{dt} = bx - ax^2y\,,$$

where $x$ and $y$ represent the concentrations of the reacting chemicals. Use `integrate.solve_ivp` to solve these equations for the case $a=1$, $b=3$ with initial conditions $x=y=0$. Calculate a solution for `t=np.linspace(0, 20, 200)`. Store the answer in a pair of arrays named `x` and `y`. Plot $x(t)$ and $y(t)$.

In [ ]:
%%graded # 3 points

# YOUR CODE HERE

In [ ]:
%%tests

_test.similar(max(x), 3.74)
_test.similar(max(y), 4.74)
_test.similar(x[:5], [0.00, 0.0828, 0.138, 0.175, 0.200])
_test.similar(y[:5], [0.00, 0.0133, 0.0471, 0.0946, 0.151])
_test.plot_shown()

# The Ways ODE Solvers Fail

In [ ]:
%video zlZy_Kkyat4

Summary:

- Besides RK45 (Runge-Kutta), other methods you may want to use are BDF which uses the backwards difference formula (discussed in the upcoming PDE lecture), and  Radau, which is similar to BDF, but is higher-order (and so is usually faster for given accuracy).
- Stiff equations have a region that requires incredibly tiny step-size. Implicit methods (BDF and Radau) can handle these well.
- Runge-Kutta doesn't conserve mechanical energy. The accumulation of errors tends to consistently increase or decrease the energy of a system, leading to unphysical results. The solution is to use change the `method`, and/or set a smaller `rtol`.
- Using BDF or Radau where they aren't needed will make your problem take much longer to run than if you used Runge-Kutta.

# Additional Resources

- Textbook section 8.5